# Putting data in kafka

In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
from json import dumps
from kafka import KafkaProducer
import pandas as pd
import json

In [6]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [7]:
data=pd.read_csv('airlines1.csv')
data

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,0,1998,1,1,2,5,1998-01-02,NW,19386,NW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2009,2,5,28,4,2009-05-28,FL,20437,FL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2013,2,6,29,6,2013-06-29,MQ,20398,MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2010,3,8,31,2,2010-08-31,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2006,1,1,15,7,2006-01-15,US,20355,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,49996,1999,4,10,12,2,1999-10-12,WN,19393,WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,49997,2010,4,11,1,1,2010-11-01,OH,20417,OH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49998,49998,2006,2,4,30,7,2006-04-30,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49999,49999,2009,3,7,8,3,2009-07-08,F9,20436,F9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df=data[['Unnamed: 0','Year','Reporting_Airline']]

In [10]:
df

,Unnamed: 0,Year,Reporting_Airline
0,0,1998,NW
1,1,2009,FL
2,2,2013,MQ
3,3,2010,DL
4,4,2006,US
...,...,...,...
49996,49996,1999,WN
49997,49997,2010,OH
49998,49998,2006,UA
49999,49999,2009,F9


In [11]:
df.rename(columns = {'Unnamed: 0':'id'}, inplace = True)

/var/folders/7r/x4_8ch8j5w547gls5xdcg2_40000gn/T/ipykernel_7364/3549953925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'Unnamed: 0':'id'}, inplace = True)


In [13]:
df.head(5)

,id,Year,Reporting_Airline
0,0,1998,NW
1,1,2009,FL
2,2,2013,MQ
3,3,2010,DL
4,4,2006,US


In [15]:
dict_data=df.to_dict('records')

In [16]:
dict_data[0]

{'id': 0, 'Year': 1998, 'Reporting_Airline': 'NW'}

In [17]:
for e in range(1000):
    producer.send("airline-topic", value=dict_data[e],key=json.dumps(dict_data[e]["id"]).encode('utf-8'))

# Reading data from batches in spark

In [29]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [19]:
#import library 
import os
from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

sc = SparkSession.builder.appName('Pyspark_kafka_airline_read_write').getOrCreate()

df = sc \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "airline-topic") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load() \
    .select("value") \
    .selectExpr("CAST(value AS STRING) as json")


23/02/23 14:13:14 WARN Utils: Your hostname, ashwins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
23/02/23 14:13:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/ashwinpandey/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ashwinpandey/.ivy2/cache
The jars for the packages stored in: /Users/ashwinpandey/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7519f581-728d-40bc-ae56-142a5c04987d;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-loggi

23/02/23 14:16:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [20]:
df.show()

+--------------------+
|                json|
+--------------------+
|{"id": 0, "Year":...|
|{"id": 1, "Year":...|
|{"id": 2, "Year":...|
|{"id": 3, "Year":...|
|{"id": 4, "Year":...|
|{"id": 5, "Year":...|
|{"id": 6, "Year":...|
|{"id": 7, "Year":...|
|{"id": 8, "Year":...|
|{"id": 9, "Year":...|
|{"id": 10, "Year"...|
|{"id": 11, "Year"...|
|{"id": 12, "Year"...|
|{"id": 13, "Year"...|
|{"id": 14, "Year"...|
|{"id": 15, "Year"...|
|{"id": 16, "Year"...|
|{"id": 17, "Year"...|
|{"id": 18, "Year"...|
|{"id": 19, "Year"...|
+--------------------+
only showing top 20 rows



In [21]:
jsonSchema = StructType([StructField("id", StringType(), True), StructField("Year", StringType(), True),
                                     StructField("Reporting_Airline", StringType(), True)])

In [22]:
# Parsing and selecting the right column data
df = df.withColumn("jsonData", from_json(col("json"), jsonSchema)) \
                .select("jsonData.*")

In [23]:
df.show()

+---+----+-----------------+
| id|Year|Reporting_Airline|
+---+----+-----------------+
|  0|1998|               NW|
|  1|2009|               FL|
|  2|2013|               MQ|
|  3|2010|               DL|
|  4|2006|               US|
|  5|1995|               DL|
|  6|2006|               CO|
|  7|2019|               9E|
|  8|2008|               YV|
|  9|2018|               WN|
| 10|1991|               US|
| 11|2014|               WN|
| 12|1994|               AA|
| 13|2013|               OO|
| 14|2003|               UA|
| 15|1988|               PI|
| 16|2007|               NW|
| 17|2015|               AS|
| 18|2006|               UA|
| 19|2017|               WN|
+---+----+-----------------+
only showing top 20 rows



# Transformation on Data

In [24]:
df.select('Reporting_Airline').groupBy('Reporting_Airline').count().show()

+-----------------+-----+
|Reporting_Airline|count|
+-----------------+-----+
|               UA|   84|
|               EA|    4|
|               PI|    7|
|               NK|    2|
|               PS|    1|
|               AA|  123|
|               NW|   52|
|               EV|   27|
|               B6|   17|
|               HP|   19|
|               TW|   17|
|               DL|  148|
|               OO|   60|
|               F9|    8|
|               YV|   15|
|               TZ|    1|
|               US|   82|
|               MQ|   47|
|               OH|    9|
|               HA|    7|
+-----------------+-----+
only showing top 20 rows



In [34]:
# how many aircrafts are there after 2015
df_filtered = df.select('id','Year','Reporting_Airline').filter('Year >= 2015')

In [35]:
df_filtered.show()

+---+----+-----------------+
| id|Year|Reporting_Airline|
+---+----+-----------------+
|  7|2019|               9E|
|  9|2018|               WN|
| 17|2015|               AS|
| 19|2017|               WN|
| 23|2020|               YV|
| 30|2019|               YX|
| 31|2019|               OH|
| 33|2016|               WN|
| 38|2018|               UA|
| 43|2015|               WN|
| 45|2017|               WN|
| 59|2017|               WN|
| 63|2019|               WN|
| 66|2015|               B6|
| 70|2016|               B6|
| 71|2016|               HA|
| 75|2016|               AA|
| 89|2017|               AA|
| 93|2016|               WN|
|101|2019|               DL|
+---+----+-----------------+
only showing top 20 rows



# Putting data back to kafka

In [36]:
query = df_filtered.selectExpr("CAST(id AS STRING) AS key", "to_json(struct(*)) AS value")\
                .write \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "localhost:9092") \
                .option("topic", "airline-sink") \
                .option("checkpointLocation", "./check") \
                .save()

23/02/23 15:03:07 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 3 : {airline-sink=LEADER_NOT_AVAILABLE}
23/02/23 15:03:07 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 4 : {airline-sink=LEADER_NOT_AVAILABLE}
23/02/23 15:03:07 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 5 : {airline-sink=LEADER_NOT_AVAILABLE}
23/02/23 15:03:07 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 6 : {airline-sink=LEADER_NOT_AVAILABLE}
